<a href="https://colab.research.google.com/github/tintubiel/MSU_ML_MFK/blob/main/Bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bootstrap

Как известно, для формирования случайного леса необходимо разнообразие каждого из решающих деревьев в его составе. Добиться такого разнообразия можно как при помощи варьирования параметров деревьев, так и при помощи варьирования обучающих выборок. Это обычно делается при помощи комбинирования методов случайных подпространств и беггинга. Метод случайных подпространств состоит в случайном выборе подмножества признаков. Бэггинг - это техника отбора подмножества объектов из исходной выборки, которая состоит в последовательном случайном выборе объектов выборки **с возвращением**.

Для ясности приведём следующий пример:

Пусть  наша выборка - это черный мешок с пронумерованными шарами. Каждый шар символизирует некоторый объект нашей выборки с соответствующим номером. Процедура бэггинга предлагает нам последовательно хорошо перемешав шары в мешке вытаскивать их один за другим не глядя, записывать их номера на лист бумаги, а затем возвращать их назад в мешок, повторяя эту операцию столько раз, сколько объектов содержится в нашей выборке. Затем мы вынем из мешка все те шары, номер которых хотя бы раз возник в нашем списке. Понятно, что скорее всего среди этих номеров будут повторяющиеся, то есть какие-то шары мы вытянем несколько раз, а какие-то - вообще ни разу. Именно поэтому наша подвыборка и получится случайной. Статистически обоснована оценка того, какой процент шаров из исходной выборки в среднем попадёт в итоговую подвыборку. Эта оценка приблизительно равна $62\%$.

Ваша задача - написать реализацию трёх функций и объединить их в класс `sample`, возвращающий по выборке некоторую случайную подвыборку, пригодную для обучения  одного из деревьев в ансамбле случайного леса.

**Замечание:** обратите внимание, что объекты в итоговой подвыборке не должны дублироваться. Мы предлагаем Вам ознакомиться с функциями [np.random.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) и [np.unique](https://numpy.org/doc/stable/reference/generated/numpy.unique.html), они могут оказаться полезны при выполнении этого задания. 

Подробнее об этом методе можно прочитать в нашей [лекции](https://colab.research.google.com/drive/1LrqEyfmOKJQdvgxZ56qPcJ_YjJFnP_Ka#scrollTo=kYjii_stqfJo) 


In [45]:
import numpy as np
np.random.seed(42)

class sample(object):
  def __init__(self, X, n_subspace):
    self.idx_subspace = self.random_subspace(X, n_subspace)
  
  def __call__(self, X, y):
    idx_obj = self.bootstrap_sample(X)
    X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
    return X_sampled, y_sampled

  @staticmethod
  def bootstrap_sample(X, random_state=42):
    """
    Заполните тело этой функции таким образом, чтобы она возвращала массив индексов выбранных при помощи бэггинга индексов.
    Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=42
    """
    idx_obj = np.unique(np.random.choice(range(len(X)), size = len(X), replace = True))
    return idx_obj

  @staticmethod
  def random_subspace(X, n_subspace, random_state=42):
    """
    Заполните тело этой функции таким образом, чтобы она возвращала массив индексов выбранных при помощи метода случайных подпространств признаков
    Количество этих признаков передается при помощи аргумента n_subspace
    Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=42
    """
    idx_subspace = np.random.choice(range(len(X[0])), size = n_subspace, replace = False)
    return idx_subspace

  @staticmethod
  def get_subsample(X, y, idx_subspace, idx_obj):
    """
    Заполните тело этой функции таким образом, чтобы она возвращала подвыборку x_sampled, y_sampled 
    по значениям индексов признаков(idx_subspace) и объектов(idx_obj) , которые должны в неё попасть
    """
    y_sampled = y[idx_obj]
    i= np.array(idx_obj)
    x_sampled = X[i][:,idx_subspace]
    return x_sampled, y_sampled

# Пример формата входных и выходных данных

In [46]:
X = np.array([[1,2,3], [4,5,6], [7,8,9]])
Y = np.array([1, 2, 3])
s = sample(X, 2)

bootstrap_indices = s.bootstrap_sample(X)
X_sampled, y_sampled = s.get_subsample(X, Y, s.idx_subspace, bootstrap_indices)

In [47]:
# Строки, выбранные из исходного массива X
bootstrap_indices

array([0, 2])

In [48]:
# Столбцы, выбранные из исходного массива X
s.idx_subspace

array([0, 1])

In [49]:
# Матрица на выходе
X_sampled

array([[1, 2],
       [7, 8]])

In [50]:
# Вектор ответов на выходе
y_sampled

array([1, 3])